In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt

from lava.proc.io.sink import RingBuffer as SinkBuffer

from lava.proc.lif.process import LIF
from lava.proc.dense.process import Dense
from lava.proc.monitor.process import Monitor

from lava.lib.dnf.connect.reshape_int.process import ReshapeInt
from lava.lib.dnf.connect.reshape_bool.process import ReshapeBool

from lava.magma.core.run_configs import RunConfig, Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

from semd.proc.tde.process import TDE
from semd.proc.reshape_conv.process import ReshapeConv
from semd.proc.change_dim.process import ChangeDim

from utils import gen_weights

events_range = np.loadtxt("../../data_processed/events_refact.txt")
# this data is already filtered by time, no need to redo it here
events = events_range

patch_center = (90, 120)
patch_size = 180
x_lim = (patch_center[1] - int(patch_size / 2), patch_center[1] + int(patch_size / 2) - 1)
y_lim = (patch_center[0] - int(patch_size / 2), patch_center[0] + int(patch_size / 2) - 1)
print(x_lim)

events_filtered = events_range
events_filtered = events_filtered[
    np.logical_and((x_lim[0] <= events_filtered[:, 1]), (events_filtered[:, 1] <= x_lim[1]))]
events_filtered = events_filtered[
    np.logical_and((y_lim[0] <= events_filtered[:, 2]), (events_filtered[:, 2] <= y_lim[1]))]
# events_filtered = events_filtered[[(y_lim[0] >= events_range[:,2]) & (events_range[:,2] <= y_lim[1])]
events_filtered = np.array(events_filtered)
events = events_filtered

events[:,1] = (events[:,1] - patch_center[1] + int(patch_size / 2)).astype(int)
events[:,2] = (events[:,2] - patch_center[0] + int(patch_size / 2)).astype(int)

t_start = events[0, 0]
duration = events[-1, 0] - events[0, 0]

# prepare event data for input to process using lava.io.source
timesteps = 50
steps = 50
width = patch_size
height = patch_size

result = np.zeros((height, width, timesteps), dtype=int)

for e in events:
    x = int(e[1])
    y = int(e[2])

    #     x = int(e[1]) - patch_center[1] + int(patch_size / 2)
    #     y = int(e[2]) - patch_center[0] + int(patch_size / 2)
    time = int((float(e[0]) - t_start) / duration * timesteps) - 1

    if e[3] == 1:
        continue
        # consider only positive events
    result[y, x, time] = 1.0

# input layer
input_n = RingBuffer(result)

shape = (height, width)
conv_shape = (2, 2)
conv_stride = (2, 2)
thresh_percent = 0.5
bias_weight = 10 / conv_shape[0]*conv_shape[1] * thresh_percent

out_shape = (int(shape[0] / conv_stride[0]), int(shape[1] / conv_stride[1]))

conv = ReshapeConv(input_shape=shape, conv_shape=conv_shape, conv_stride=conv_stride, bias_weight=bias_weight)
input_n.s_out.connect(conv.s_in)

lif = LIF(shape=out_shape, du=1.0, dv=0.0, vth=10.0)
conv.s_out.connect(lif.a_in)

detector_shape = (out_shape[0], out_shape[1] - 1)
left_detector = TDE(shape=detector_shape, du=(0.05))

change_dim_excit = ChangeDim(shape=(out_shape), col_del=[1], row_del=[])
change_dim_trig = ChangeDim(shape=(out_shape), col_del=[-1], row_del=[])

lif.s_out.connect(change_dim_excit.s_in)
lif.s_out.connect(change_dim_trig.s_in)

change_dim_trig.s_out.connect(left_detector.t_in)
change_dim_excit.s_out.connect(left_detector.a_in)

output_n = SinkBuffer(shape=detector_shape, buffer=steps)
left_detector.s_out.connect(output_n.a_in)

output_lif = SinkBuffer(shape=out_shape, buffer=steps)
lif.s_out.connect(output_lif.a_in)

output_n_in = SinkBuffer(shape=shape, buffer=steps)
input_n.s_out.connect(output_n_in.a_in)

monitor = Monitor()
monitor.probe(left_detector.u, num_steps=steps)

rcfg = Loihi1SimCfg(select_tag='floating_pt', select_sub_proc_model=True)

for i in range(steps):
    run_condition = RunSteps(num_steps=1)
    input_n.run(condition=run_condition, run_cfg=rcfg)

data = output_n.data.get()
data_in = output_n_in.data.get()
data_lif = output_lif.data.get()
data_u = monitor.get_data()
input_n.stop()

ImportError: cannot import name 'gen_weights' from 'utils' (C:\Users\schiavaz\lava-nc\depth\semd_tests\semd_refact\../..\utils.py)

In [ ]:
plt.figure(figsize=(10, 10))
m = plt.imshow(np.delete(np.sum(data, axis=-1), 0, 1), cmap="jet")
plt.colorbar(m)

In [ ]:
data_sum = np.sum(data[:,:,:], axis=-1)
data_sum = np.delete(data_sum, 0, 1)
data_masked = np.ma.masked_where(np.logical_or(data_sum == 0, data_sum > 9.0), data_sum)
plt.figure(figsize=(10, 10))
m = plt.imshow(data_masked, cmap="jet")
plt.colorbar(m)

In [ ]:
np.unravel_index(data_sum.argmax(), data_sum.shape)

In [ ]:
data_u = data_u["Process_3"]["u"]

In [ ]:
data_u

In [ ]:
data_sum = np.sum(data_u, axis=0)
np.unravel_index(data_sum.argmax(), data_sum.shape)

In [ ]:
plt.imshow(np.sum(data_u[:,:,:], axis=0))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.sum(data_in[:,:,:], axis=-1))

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.sum(data_lif, axis=-1))

In [ ]:
plt.imshow(data[:,:,30])